#### Busca os links no youtube usando um gerador de strings com a estratégia PICO

In [ ]:
from Funcs.tratamentoDeVideo import get_youtube_video_links
from Funcs.db import push_links_into_db

#video_links = {}
MAX_RESULT = 2


def busca_pico(p, i, c, o):
    cont = 0
    for individo in p:
        for method in i:
            for object in c:
                for effect in o:
                    print(f'{individo} {method} {object} {effect}')
                    try:
                        links = get_youtube_video_links(f'{individo} {method} {object} {effect}',MAX_RESULT)
                        cont += 1
                        print(cont)
                        # Salva os links gerados na tabela de links do db videos_data
                        push_links_into_db('videos_data.db','links',links)
                    except Exception as e:
                        print(f"<><><> Erro ao buscar link no youtube: {e}")
                        return



#### Faz a busca dos links no youtube em Inglês

In [ ]:
population = ['Teach', 'Science', 'Technology', 'Engineer', 'Mathematics']
intervention = ['Microcontrollers', 'STEM Maker', 'Sensors', 'Do it your self']
comparison = ['STEM Maker','Traditional teaching method']
outcome = ['']

busca_pico(population, intervention, comparison, outcome)

#### Faz a busca dos links no youtube em Português

In [ ]:
population = ['Ensinar', 'Ciência', 'Tecnologia', 'Engenharia', 'Matemática']
intervention = ['STEM Maker', 'Microcontroladores', 'Sensores', 'Faça você mesmo']
comparison = ['STEM Maker','Método tradicional de ensino']
outcome = ['']

busca_pico(population, intervention, comparison, outcome)

#### Faz buscas menores, apenas para teste

In [ ]:
population = ['Ensinar', 'Teach']
intervention = ['Microcontroladores']
comparison = ['STEM Maker','Método tradicional de ensino']
outcome = ['']

#busca_pico(population, intervention, comparison, outcome)

#### Faz o download do vídeo
> Para retirada da transcrição do audio, screenshots e descritor das screenshots

In [ ]:
from Funcs.tratamentoDeVideo import download_video
from Funcs.db import fetch_links_from_db
import asyncio
from concurrent.futures import ThreadPoolExecutor

async def download_videos_concurrently():
    links = fetch_links_from_db('videos_data.db', 'links')
    
    # Usar ThreadPoolExecutor para baixar vídeos em paralelo
    with ThreadPoolExecutor(max_workers=12) as executor:  # max_workers define o número de threads
        loop = asyncio.get_event_loop()
        tasks = []
        
        for url in links:
            tasks.append(loop.run_in_executor(executor, download_video, url))
        
        # Aguardar a conclusão de todos os downloads
        await asyncio.gather(*tasks)

if __name__ == '__main__':
    try:
        asyncio.run(download_videos_concurrently())
    except RuntimeError:
        # Caso o erro aconteça, significa que estamos em um loop de eventos ativo
        # Nesse caso, usaremos o loop de eventos atual
        loop = asyncio.get_event_loop()
        loop.create_task(download_videos_concurrently())



#### Extrai o audio do vídeo

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from Funcs.tratamentoDeAudio import extract_audio_from_video
from Funcs.getPaths import get_Paths

async def extract_audio_concurrently():
    # Obtém os caminhos dos vídeos
    paths = get_Paths("Videos")
    
    # Usar ThreadPoolExecutor para rodar a extração de áudio em paralelo
    with ThreadPoolExecutor(max_workers=4) as executor:  # Defina max_workers com base no número de threads que você quer
        loop = asyncio.get_event_loop()
        tasks = []
        
        for path in paths:
            id = path.split("\\")[-1]
            video_path = f'{path}\\{id}.mp4'
            audio_path = f'{path}\\Audio\\{id}.wav'
            tasks.append(loop.run_in_executor(executor, extract_audio_from_video, video_path, audio_path))
        
        # Aguarda a conclusão de todas as extrações de áudio
        await asyncio.gather(*tasks)

if __name__ == '__main__':
    # Verifica se já existe um loop de eventos
    try:
        # Tenta rodar no loop de eventos atual (caso já exista)
        asyncio.run(extract_audio_concurrently())
    except RuntimeError:
        # Caso o erro aconteça, significa que estamos em um loop de eventos ativo
        # Nesse caso, usaremos o loop de eventos atual
        loop = asyncio.get_event_loop()
        loop.create_task(extract_audio_concurrently())


#### Particiona o audio para poder fazer as transcrições 

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from Funcs.tratamentoDeAudio import split_audio_by_size
from Funcs.getPaths import get_Paths

async def split_audio_concurrently():
    # Obtém os caminhos dos vídeos
    paths = get_Paths("Videos")
    
    # Usar ThreadPoolExecutor para rodar a divisão de áudio em paralelo
    with ThreadPoolExecutor(max_workers=12) as executor:  # Defina max_workers com base no número de threads que você quer
        loop = asyncio.get_event_loop()
        tasks = []
        
        for path in paths:
            id = path.split("\\")[-1]
            audio_path = f'{path}\\Audio\\{id}.wav'
            splits_path = f'{path}\\Audio\\Splits'
            tasks.append(loop.run_in_executor(executor, split_audio_by_size, audio_path, 9, splits_path))
        
        # Aguarda a conclusão de todas as divisões de áudio
        await asyncio.gather(*tasks)

if __name__ == '__main__':
    # Verifica se já existe um loop de eventos
    try:
        # Tenta rodar no loop de eventos atual (caso já exista)
        asyncio.run(split_audio_concurrently())
    except RuntimeError:
        # Caso o erro aconteça, significa que estamos em um loop de eventos ativo
        # Nesse caso, usaremos o loop de eventos atual
        loop = asyncio.get_event_loop()
        loop.create_task(split_audio_concurrently())


#### Realiza a transcrição de cada split de audio e salva no bd

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from queue import Queue
from Funcs.tratamentoDeAudio import transcribe_audio
from Funcs.getPaths import get_Paths
from Funcs.db import push_transcribes_into_db, get_languange
import threading

# Fila para armazenar os resultados das transcrições
results_queue = Queue()

def process_video(path):
    """Função para processar um único vídeo."""
    id = path.split("\\")[-1]
    video_transcription = ''

    for chunk in get_Paths(path + '\\Audio\\Splits'):
        video_language = get_languange('videos_data.db', 'links', id)[0]
        transcription = transcribe_audio(chunk, video_language)
        video_transcription += ' ' + transcription

    # Enviar o resultado para a fila
    results_queue.put((id, video_transcription))

def db_writer(database_path, table_name):
    """Thread para escrever os resultados no banco de dados."""
    while True:
        # Obter dados da fila
        data = results_queue.get()

        # Sair do loop quando a fila está vazia e todos os trabalhos estão concluídos
        if data is None:
            break

        # Desempacotar os dados
        id, video_transcription = data

        # Escrever no banco de dados
        push_transcribes_into_db(database_path, table_name, video_transcription, id)

def main():
    # Caminhos para processar
    video_paths = get_Paths("Videos")

    # Thread para escrita no banco de dados
    db_thread = threading.Thread(target=db_writer, args=('videos_data.db', 'transcribes'))
    db_thread.start()

    # Executor para processar vídeos em paralelo
    with ThreadPoolExecutor(max_workers=12) as executor:
        for path in video_paths:
            executor.submit(process_video, path)

    # Finalizar a thread de banco de dados
    results_queue.put(None)  # Sinalizar término
    db_thread.join()

if __name__ == "__main__":
    main()


#### Realiza as screenshots do vídeo para receberem uma caption

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from Funcs.tratamentoDeImg import make_ScreenShots
from Funcs.getPaths import get_Paths

def process_video(path):
    """Função para processar capturas de tela de um vídeo."""
    id = path.split("\\")[-1]
    make_ScreenShots(path + '/' + id + '.mp4', path + '/Screenshots', 10)

def main():
    # Obter todos os caminhos dos vídeos
    video_paths = get_Paths("Videos")

    # Usar um ThreadPoolExecutor para processar os vídeos em paralelo
    with ThreadPoolExecutor(max_workers=12) as executor:  # Ajuste max_workers conforme necessário
        executor.map(process_video, video_paths)

if __name__ == "__main__":
    main()


#### Gera uma caption para cada screenshot e salva no bd

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from Funcs.tratamentoDeImg import generate_caption
from Funcs.getPaths import get_Paths
from Funcs.db import push_caption_into_db

def process_screenshot(ss, video_id):
    """Processa uma captura de tela e retorna a legenda."""
    caption = generate_caption(ss)
    screenshot_id = ss.split("\\")[-1]
    return caption, video_id, screenshot_id

def main():
    database_path = 'videos_data.db'
    table_name = 'captions'
    video_paths = get_Paths("Videos")

    # Configurar ThreadPoolExecutor
    max_workers = 12  # Ajuste conforme o hardware disponível
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []

        # Agendar tarefas de geração de legendas
        for path in video_paths:
            video_id = path.split("\\")[-1]
            screenshot_paths = get_Paths(path + '\\Screenshots')
            for ss in screenshot_paths:
                print("Screnshot: ")
                print(ss)
                futures.append(executor.submit(process_screenshot, ss, video_id))

        # Processar resultados conforme as tarefas são concluídas
        for future in as_completed(futures):
            #print(future)
            try:
                caption, video_id, screenshot_id = future.result()
                 # Inserir no banco de dados
                push_caption_into_db(database_path, table_name, caption, video_id, screenshot_id)
            except Exception as e:
                print(f"Erro ao processar ou inserir no banco: {e}")

if __name__ == "__main__":
    main()
